In [ ]:
import os
os.chdir('..')

In [ ]:
import torch
import numpy as np

from instance_generator import sample_instances
from gnn_library.util import load
from gnn_library.train import train_meta_model
from evaluate import evaluate_model
from params import TEST_CONFIGS, REGIMES
from util import upload_meta_experiment, load_meta_experiments, \
    graph_config_to_string, _plot_meta_ratios, _plot_meta_ratios_main

%load_ext autoreload
%autoreload 2

device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')
print("PyTorch has version {}".format(torch.__version__))
print('Using device:', device)

## Train meta-GNN

In [ ]:
GNN1, args1 = load('GNN_large_6_10', device)
GNN2, args2 = load('GNN_large_10_6', device)

args = {
    'processor':         'DeeperGCN',
    'head':              'meta',
    'num_layers':        4,
    'num_mlp_layers':    2,
    'aggr':              'max',
    'batch_size':        6,
    'node_feature_dim':  7,
    'edge_feature_dim':  1,
    'graph_feature_dim': 2,
    'hidden_dim':        8,
    'output_dim':        2,
    'head_mlp_dim':      8,
    'dropout':           0,
    'epochs':            35,
    'opt':               'adam',
    'opt_scheduler':     'none',
    'opt_restart':       0,
    'weight_decay':      5e-3,
    'lr':                0.001,
    'device':            device
}

_ = train_meta_model(
    base_models=[GNN1, GNN2],
    name='META_GNN_TEST',
    args=args
)

## Regime generalization experiment

#### Compute meta-GNN CRs over all graph configurations and node ratio regimes

In [ ]:
META_GNN, args = load('META_GNN_TEST', device)

In [ ]:
seed = 0
rng = np.random.default_rng(seed)
num_trials = 50
batch_size = 100
graph_configs = TEST_CONFIGS['ALL']
node_configs = REGIMES['REGIME_GENERALIZATION']
baselines_kwargs = {
    'greedy': {},
    'greedy_t': {'threshold': 0.35},
    'lp_rounding': {'rng': rng},
    'naor_lp_rounding': {'rng': rng},
    'pollner_lp_rounding': {'rng': rng}
}

regimes = [x/y for (x,y) in node_configs]

def _init_data():
    return {
        "num_trials": num_trials,
        "meta_gnn": [],
        "greedy": [],
        "greedy_t": [],
        "lp_rounding": [],
        "naor_lp_rounding": [],
        "pollner_lp_rounding": [],
        "meta_threshold": []
    }

for graph_config in graph_configs:
    data = _init_data()
    graph_str = graph_config_to_string(graph_config)

    for i, node_config in enumerate(node_configs):
        print(graph_config, node_config)
        instances = sample_instances(
            *node_config,
            num_trials,
            rng,
            args.__dict__,
            **graph_config
        )

        rng = np.random.default_rng(seed)
        crs, _ = evaluate_model(
            meta_model=META_GNN,
            meta_model_type='gnn',
            base_models=[GNN1, GNN2],
            instances=instances,
            batch_size=batch_size,
            rng=rng,
            num_realizations=5,
            baselines=[],
            **baselines_kwargs
        )

        data['meta_gnn'].append(crs['learned'])
        data['greedy'].append(crs['greedy'])
        data['greedy_t'].append(crs['greedy_t'])
        data['lp_rounding'].append(crs['lp_rounding'])

    upload_meta_experiment(graph_str, data)

#### Generate regime generalization plots

In [ ]:

results = load_meta_experiments(graph_configs)
_plot_meta_ratios(
    regimes,
    results,
    ['meta_gnn', 'meta_threshold'],
    lambda graph_type: graph_type,
    confidence = 0.95
)